In [5]:
import pandas as pd
import numpy as np
import sqlalchemy
from urllib.parse import urlencode
import requests
from requests import Response
import datetime as dt
from datetime import time, date, timedelta
from dateutil.relativedelta import relativedelta
import itertools
from math import ceil
import scipy.stats as stats

In [20]:
from ta.momentum import KAMAIndicator
def KAMA(window_para,fast_length, slow_length, to_csv):
    result = pd.read_csv('raw-weekly-pricevolume.csv')
    codes = result['code'].unique().tolist()
    for code in codes:
        close_weekly = result[result['code']==code]['close_weekly']
        result.loc[code == result['code'].shift(slow_length),'kama'] = KAMAIndicator(close=result['close_weekly'],window=window_para, pow1=fast_length, pow2=slow_length).kama()
    result.loc[result['code'] == result['code'].shift(1),'return'] = result['close_weekly']/result['close_weekly'].shift(1) - 1
    result = result.dropna()
    result.loc[(result['code'] == result['code'].shift(1))&(result['close_weekly']>result['kama']),'signal'] = 1
    result.loc[(result['code'] == result['code'].shift(1))&(result['close_weekly']<result['kama']),'signal'] = 0
    result.loc[result['code'] != result['code'].shift(1),'signal'] = 0
    result = result.ffill()

    result.loc[result['code'] == result['code'].shift(1),'daily_return'] = result['signal'].shift(1)*result['return']
    result.loc[(result['code'] == result['code'].shift(1))&(result['signal']==1)&(result['signal'].shift(1)==0),'daily_return']=result['daily_return']-0.005


    dailyprofit = pd.pivot(result, index = 'date', columns = 'code', values = 'daily_return')
    dailyprofit = dailyprofit.replace(0, np.nan)
    dailyprofit.reset_index(inplace = True)
    day_return = dailyprofit.copy()
    day_return['count'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].count(axis = 1) #exclude date column
    day_return.loc[day_return['count']<5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)*(day_return['count']/5)
    day_return.loc[day_return['count']==0,'profit'] = 0
    day_return.loc[day_return['count']>=5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)

    profit = (day_return['profit'] +1).cumprod()
    day_return['cumulative'] = profit
    if to_csv:
        day_return.to_csv('checkkama.csv')
    cul_arr = day_return['cumulative'].to_list()
    dd_arr = []
    maxcul = 0
    for x in day_return.cumulative.dropna().tolist():
        if x >= maxcul:
            maxcul = x
            dd_arr.append(0)
        else:
            dd_arr.append((maxcul-x)/maxcul)
    print(f'MDD {max(dd_arr)}')
    return float(profit[-1:]), window_para,fast_length, slow_length

In [15]:
optimize_result = []
for window_para in range(10,30, 1):
    for fast_length in range(2,20, 1):
        for slow_length in range(10,40,2):
            if fast_length < slow_length:
                x=KAMA(window_para,fast_length, slow_length, to_csv = False)
                optimize_result.append(x)    
                print(fast_length,slow_length)
optimize_result = pd.DataFrame(optimize_result)

2 10
2 12
2 14
2 16
2 18
2 20
2 22
2 24
2 26
2 28
2 30
2 32
2 34
2 36
2 38
3 10
3 12
3 14
3 16
3 18
3 20
3 22
3 24
3 26
3 28
3 30
3 32
3 34
3 36
3 38
4 10
4 12
4 14
4 16
4 18
4 20
4 22
4 24
4 26
4 28
4 30
4 32
4 34
4 36
4 38
5 10
5 12
5 14
5 16
5 18
5 20
5 22
5 24
5 26
5 28
5 30
5 32
5 34
5 36
5 38
6 10
6 12
6 14
6 16
6 18
6 20
6 22
6 24
6 26
6 28
6 30
6 32
6 34
6 36
6 38
7 10
7 12
7 14
7 16
7 18
7 20
7 22
7 24
7 26
7 28
7 30
7 32
7 34
7 36
7 38
8 10
8 12
8 14
8 16
8 18
8 20
8 22
8 24
8 26
8 28
8 30
8 32
8 34
8 36
8 38
9 10
9 12
9 14
9 16
9 18
9 20
9 22
9 24
9 26
9 28
9 30
9 32
9 34
9 36
9 38
10 12
10 14
10 16
10 18
10 20
10 22
10 24
10 26
10 28
10 30
10 32
10 34
10 36
10 38
11 12
11 14
11 16
11 18
11 20
11 22
11 24
11 26
11 28
11 30
11 32
11 34
11 36
11 38
12 14
12 16
12 18
12 20
12 22
12 24
12 26
12 28
12 30
12 32
12 34
12 36
12 38
13 14
13 16
13 18
13 20
13 22
13 24
13 26
13 28
13 30
13 32
13 34
13 36
13 38
14 16
14 18
14 20
14 22
14 24
14 26
14 28
14 30
14 32
14 34
14 36
14 38
15 1

KeyboardInterrupt: 

In [17]:
optimize_result = pd.DataFrame(optimize_result)

In [21]:
KAMA(10,5, 16, to_csv = False)

MDD 0.19066529638393795


(3.4378686567581944, 10, 5, 16)

In [18]:
optimize_result.sort_values(by=0, ascending =False ).head(50)

,0,1,2,3
48,3.437869,10,5,16
44,3.368151,10,4,38
43,3.359139,10,4,36
42,3.305218,10,4,34
41,3.273198,10,4,32
74,3.256282,10,6,38
37,3.182146,10,4,24
40,3.163462,10,4,30
54,3.142641,10,5,28
38,3.135391,10,4,26
